In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from glob import glob
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from scipy import stats

In [30]:
mu_b = 5.7883818012 * 10**(-5)
mu_b

5.7883818012e-05

In [31]:
list_sweepVT = sorted(glob('*Voltage_Sweep.tsv.gz'))

In [32]:
list_calibration = glob('*.wlen_to_px.tsv.gz')

In [33]:
calibration = 1230.8 / (pd.read_csv(list_calibration[0], header=None).as_matrix().transpose())[0]

In [34]:
sweepVT_sigmam = np.array([pd.read_csv(file, sep='\t', header=None).as_matrix() for file in list_sweepVT[:9]])
sweepVT_sigmap = np.array([pd.read_csv(file, sep='\t', header=None).as_matrix() for file in list_sweepVT[9:]])

In [35]:
np.shape(sweepVT_sigmam)


(9, 81, 1340)

In [36]:
np.shape(sweepVT_sigmap)

(9, 81, 1340)

In [37]:
np.shape(sweepVT_sigmap[0,0,:])

(1340,)

In [38]:
spec = sweepVT_sigmap[0,15,:]
calibration

array([1.85623947, 1.85587281, 1.85550909, ..., 1.47222415, 1.47199701,
       1.47177171])

In [39]:
output_notebook()

Loading BokehJS ...

In [40]:
def lorentz(x, x_n, G):
    X = (x_n - x)/(G/2)
    return 1/(1+X**2)

In [41]:
a_min, a_max = 1.69, 1.72

In [42]:
def fit_a(data,
          cal=calibration,
          a_min=1.695,
          a_max=1.72,
          base_function=lorentz,
          p0=[1.70, 0.05]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(lorentz, cal_slice, data_slice, p0=p0)
    return popt[0]  

In [69]:
#a_energies_sp = np.array([fit_a(field) for field in sweepVT_sigmap[:,23,:]])
#a_energies_sm = np.array([fit_a(field) for field in sweepVT_sigmam[:,23,:]])
#g_factors = (a_energies_sm - a_energies_sp)/(mu_b * np.arange(9))
#a_energies_sp
a_energies_sp = np.array([fit_a(field) for field in sweepVT_sigmap[4,:,:]])
a_energies_sm = np.array([fit_a(field) for field in sweepVT_sigmam[4,:,:]])
#g_factors = (a_energies_sm - a_energies_sp)/(mu_b * np.arange(9))
a_energies_sp

/usr/lib/python3/dist-packages/scipy/optimize/minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


array([ 1.67335382e+00, -1.47572585e+04,  1.63251661e+00,  1.64176146e+00,
        1.62831003e+00,  1.54731114e+00, -1.37145693e+05, -5.46172129e+05,
       -1.57750478e+06, -1.03474717e+06, -6.22791475e+05,  1.71055525e+00,
        1.83130696e+00,  1.70737146e+00,  1.70704265e+00,  1.70693085e+00,
        1.70684608e+00,  1.70679988e+00,  1.70678896e+00,  1.70679972e+00,
        1.70682118e+00,  1.70671474e+00,  1.70677835e+00,  1.70673637e+00,
        1.70674102e+00,  1.70676507e+00,  1.70674803e+00,  1.70678533e+00,
        1.70674927e+00,  1.70680971e+00,  1.70674191e+00,  1.70677873e+00,
        1.70693886e+00,  1.70707511e+00,  1.70713363e+00,  1.70730939e+00,
        1.70742872e+00,  1.70753565e+00,  1.70774380e+00,  1.70787106e+00,
        1.70802100e+00,  1.70833261e+00,  1.70845688e+00,  1.81553279e+00,
        1.70898484e+00,  1.70967006e+00,  1.71027529e+00,  1.71106069e+00,
        1.71182432e+00,  1.71353207e+00, -8.00842320e+05, -1.28249902e+06,
       -2.13937745e+06, -

In [75]:
p = figure(y_range=(1.705, 1.708))
xxx = np.arange(0,81,1)
p.scatter(xxx, a_energies_sm)
p.scatter(xxx, a_energies_sp, color='red')
show(p)

In [53]:
p = figure()
slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(9) * mu_b, a_energies_sm)
teslas = np.arange(9) * mu_b
linear = slope * teslas + intercept
p.scatter(teslas, a_energies_sm)
p.line(mu_b * np.arange(9), linear, color='red')
print(slope)
#plt.plot(calibration, y)
show(p)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [45]:
sweepVT_sigmap[:,15,:]

array([[-5, -1,  0, ...,  7,  4,  5],
       [-3, -2,  2, ...,  2,  1,  5],
       [-4, -1,  1, ...,  2,  3,  1],
       ...,
       [-3,  1,  2, ...,  3,  3,  2],
       [-1, -1,  1, ...,  0,  1,  2],
       [-3, -2,  1, ...,  0,  3,  0]])

In [46]:
p = figure()
p.line(calibration, spec)
show(p)